In [ ]:
%%capture
! pip install accelerate peft transformers datasets bitsandbytes
#런타임 다시 시작

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig
import torch

model_name = "kfkas/Llama-2-ko-7b-Chat"
dataset_name = "mlabonne/guanaco-llama2-1k"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type="CAUSAL_LM"
)

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
getattr(torch, "float16")

torch.float16

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "kfkas/Llama-2-ko-7b-Chat",
    quantization_config=bnb_config,
    device_map='auto'
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

trainable params: 33,554,432 || all params: 6,889,410,560 || trainable%: 0.4870435824338505


In [ ]:
model.save_pretrained("llama-2-ko-7b-chat-guanaco")

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

peft_model = "/content/llama-2-ko-7b-chat-guanaco"
base_model = AutoModelForCausalLM.from_pretrained(
    "kfkas/Llama-2-ko-7b-Chat",
    quantization_config=bnb_config,
    device_map='auto'
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(base_model, peft_model)
tokenizer = AutoTokenizer.from_pretrained("kfkas/Llama-2-ko-7b-Chat")

model = model.to("cuda")
model.eval()

tokenizer_config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(46336, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, b

In [ ]:
prompt = "What is Hacktoberfest?"
inputs = tokenizer(f"<s>[INST] {prompt} [/INST]", return_tensors="pt")
with torch.no_grad():
  outputs = model.generate(
      input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100
  )
  print(
      tokenizer.batch_decode(
          outputs.detach().cpu().numpy(), skip_special_tokens=True
      )[0]
  )

[INST] What is Hacktoberfest? [/INST]Hacktoberfest는 2014년 10월 24일, Red Hat의 개발자이자 오픈소스 소프트웨어 옹호자인 Matt Abley가 제안한 행사입니다. 그는 10월 24일을 Hacktivism의 날로 지정하고, 10월 24일 동안 10개의 오픈소스 프로젝트에 기여하는 것을 목표로 하는 Hacktivism 주간을 만들었습니다. 이 행사는 20
